<a href="https://colab.research.google.com/github/mohamedelsehetryfehu/ml-easy/blob/main/ml_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Prophet

In [ ]:
# !pip install pystan==2.19.1.1
# !pip install convertdate
# !pip install lunarcalendar
# !pip install holidays==0.10.5
# !pip install fbprophet

In [ ]:
from datetime import datetime


In [ ]:
from datetime import datetime

test_case = list(('te', 'r'))

# Get current timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# Construct filename with timestamp
filename = f"cv_easy_{timestamp}.txt"
# File Saving
with open(filename, 'w') as file:
    # Serialize and write the tuple to the file
    file.write(str(list(test_case)))

In [ ]:
# !pip install pystan~=2.14
# !pip install fbprophet

!pip install prophet

import prophet

In [ ]:
import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error

# Function to calculate Root Mean Square Error (RMSE)
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

# Load the historical time series data
# Assuming 'attacks_data' contains the historical attacks data with daily intervals for 500 days
# It should be in the form of a list or numpy array
# For demonstration, let's assume attacks_data is a list of daily attacks for 500 days
#attacks_data = [...]  # Replace [...] with your actual data

attacks_data = pd.read_csv('series_data.csv')  # Replace [...] with your actual data




# Create a DataFrame with the time series data
df = pd.DataFrame({'ds': pd.date_range(start='2015-07-01', periods=len(attacks_data), freq='D'),
                   'y': attacks_data[attacks_data.columns[1]]})

# Handle missing values by linear interpolation
df['y'].interpolate(method='linear', inplace=True)

# Detect and handle outliers
outliers = df['y'].rolling(window=7, center=True).median()  # Median of a rolling window
df['y'] = np.where(abs(df['y'] - outliers) > 3 * outliers.std(), outliers, df['y'])

# Define the Prophet model
prophet_model = Prophet()

# Add outlier regressors
df['outlier'] = np.where((df['y'] - outliers).abs() > 3 * outliers.std(), 1, 0)
df['outlier'] = df['outlier'].fillna(0)  # Fill NaN with 0

# Add outlier regressor to Prophet model
prophet_model.add_regressor('outlier')

# Fit the model
prophet_model.fit(df)

# Forecast the attacks for the next 50 days
future = prophet_model.make_future_dataframe(periods=50)
future['outlier'] = 0  # Assuming no outliers in the future
forecast = prophet_model.predict(future)

# Extracting forecasted values for the next 50 days
forecasted_attacks = forecast.tail(50)['yhat'].values

# Print the forecasted attacks
print("Forecasted attacks for the next 50 days:")
print(forecasted_attacks)

# Evaluate the model using RMSE
# Assuming you have actual values for the next 50 days stored in 'actual_attacks'
actual_attacks = [2.0, 12.0, 13.0, 1.0, 10.0, 8.0, 6.0, 6.0, 6.0, 8.0, 9.0, 6.0, 1.0, 12.0, 11.0, 7.0, 5.0, 12.0, 8.0, 7.0, 15.0, 11.0, 9.0, 7.0, 9.0, 6.0, 8.0, 13.0, 8.0, 5.0, 12.0, 10.0, 7.0, 7.0, 11.0, 7.0, 14.0, 11.0, 3.0, 10.0, 5.0, 14.0, 8.0, 11.0, 8.0, 24.0, 10.0, 15.0, 12.0, 11.0] # Replace [...] with actual values for the next 50 days

# Calculate RMSE
rmse = calculate_rmse(actual_attacks, forecasted_attacks)
print("Root Mean Square Error (RMSE):", rmse)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmplbcmg7je/y4xx6md8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmplbcmg7je/ebp3fsch.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=14625', 'data', 'file=/tmp/tmplbcmg7je/y4xx6md8.json', 'init=/tmp/tmplbcmg7je/ebp3fsch.json', 'output', 'file=/tmp/tmplbcmg7je/prophet_model7gbrh5gw/prophet_model-20240307195423.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
19:54:23 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
19:54:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Forecasted attacks for the next 50 days:
[8.53495795 8.67444828 7.86234096 8.00184932 7.94901255 8.49999046
 8.19631111 8.53218658 8.67167691 7.85956959 7.99907795 7.94624118
 8.49721909 8.19353974 8.52941521 8.66890554 7.85679822 7.99630658
 7.94346981 8.49444772 8.19076837 8.52664384 8.66613417 7.85402685
 7.99353521 7.94069844 8.49167635 8.187997   8.52387247 8.6633628
 7.85125548 7.99076384 7.93792707 8.48890498 8.18522563 8.5211011
 8.66059143 7.84848411 7.98799247 7.9351557  8.48613361 8.18245426
 8.51832973 8.65782006 7.84571274 7.9852211  7.93238433 8.48336224
 8.17968289 8.51555836]
Root Mean Square Error (RMSE): 4.088910615624118


#SES Method

In [ ]:
import numpy as np
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
from scipy.signal import medfilt

# Function to calculate Root Mean Square Error (RMSE)
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

# Load the historical time series data
# Assuming 'attacks_data' contains the historical attacks data with daily intervals for 500 days
# It should be in the form of a list or numpy array
# For demonstration, let's assume attacks_data is a list of daily attacks for 500 days

data = pd.read_csv('series_data.csv')  # Replace [...] with your actual data

selected_data = data['visits'].astype(np.float32)

#selected_data = data['visits'][np.random.choice(data['visits'].shape[0], size=400, replace=False), :]

# Handle missing values using interpolation
attacks_data = np.array(selected_data)
missing_indices = np.isnan(attacks_data)
if np.any(missing_indices):
    attacks_data[missing_indices] = np.interp(np.flatnonzero(missing_indices),
                                              np.flatnonzero(~missing_indices),
                                              attacks_data[~missing_indices])

# Handle outliers using median filtering
attacks_data = medfilt(attacks_data, kernel_size=5)  # Adjust kernel size as needed



# Define the SES model
ses_model = SimpleExpSmoothing(attacks_data)

# Fit the model
ses_fit = ses_model.fit()

# Forecast the attacks for the next 50 days
forecasted_attacks = ses_fit.forecast(steps=50)

# Print the forecasted attacks
print("Forecasted attacks for the next 50 days:")
print(forecasted_attacks)

# Evaluate the model using RMSE
# Assuming you have actual values for the next 50 days stored in 'actual_attacks'
actual_attacks = [2.0, 12.0, 13.0, 1.0, 10.0, 8.0, 6.0, 6.0, 6.0, 8.0, 9.0, 6.0, 1.0, 12.0, 11.0, 7.0, 5.0, 12.0, 8.0, 7.0, 15.0, 11.0, 9.0, 7.0, 9.0, 6.0, 8.0, 13.0, 8.0, 5.0, 12.0, 10.0, 7.0, 7.0, 11.0, 7.0, 14.0, 11.0, 3.0, 10.0, 5.0, 14.0, 8.0, 11.0, 8.0, 24.0, 10.0, 15.0, 12.0, 11.0]  # Replace [...] with actual values for the next 50 days

# Calculate RMSE
rmse = calculate_rmse(actual_attacks, forecasted_attacks)
print("Root Mean Square Error (RMSE):", rmse)

#LSTM

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from keras.models import load_model
from keras.models import Sequential
from keras.layers import LSTM, Dense
import os

In [ ]:
import numpy as np
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error
from scipy.signal import medfilt

In [ ]:
# Replace 'path/to/your/folder' with the actual path within your Drive
os.chdir('/content/drive/MyDrive/Dell/Dataset')

In [ ]:
def evaluate_forecast(y_true, y_pred):

    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

In [ ]:
# Read the dataset
data = pd.read_csv('series_data.csv')

In [ ]:
# Data preprocessing
# Convert timestamp to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [ ]:
# Feature Engineering
# Extract relevant features
data['day_of_week'] = data['timestamp'].dt.dayofweek
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day

In [ ]:
data['visits'] = data['visits'].astype(np.float32)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
data['visits'] = scaler.fit_transform(data['visits'].values.reshape(-1,1))

In [ ]:
# Prepare data for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 30
X, y = create_sequences(data['visits'].values, seq_length)

In [ ]:
# Split the data into train and test sets to match the forecast horizon
split = len(data) - 50
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
X_train.shape

(450, 30)

In [ ]:
input_shape=(seq_length, 1)
input_shape

(30, 1)

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32)

In [ ]:
# Forecasting
forecast = []
current_batch = X_test[0].reshape((1, seq_length, 1))
print(current_batch.shape)


for i in range(50):  # Iterate 50 times for 50 days forecast
    current_pred = model.predict(current_batch)[0]
    forecast.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
# Inverse transform the forecasted values
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1,1))

In [ ]:
# Ensure y_test_trimmed contains the next 50 days of attack data
y_test_trimmed = data['visits'][-50:].values.reshape(-1, 1)

In [ ]:
# Evaluate the forecast
rmse = evaluate_forecast(y_test_trimmed, forecast)
print("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 8.93589416162554


In [ ]:
forecast_flat = [item for sublist in forecast for item in sublist]

print("Forecast for the next 50 days:")
print(forecast_flat)

Forecast for the next 50 days:
[8.994633, 9.012736, 9.027131, 9.039193, 9.04888, 9.057257, 9.064123, 9.070072, 9.074838, 9.07904, 9.082726, 9.085924, 9.088574, 9.090654, 9.092623, 9.094451, 9.095683, 9.096578, 9.097241, 9.09813, 9.099025, 9.09956, 9.100189, 9.100512, 9.100839, 9.101118, 9.101398, 9.101806, 9.101876, 9.101847, 9.102051, 9.102179, 9.102289, 9.102383, 9.102463, 9.102532, 9.1025915, 9.102641, 9.102686, 9.102722, 9.102756, 9.102781, 9.102806, 9.102826, 9.102843, 9.102859, 9.102872, 9.102882, 9.102891, 9.1029]


In [ ]:
model.save('lstm_attack_forecast_model_mleasy.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#Convert to Tensorflow Lite

In [ ]:
!pip install tensorflow-cpu

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
new_model = tf.keras.models.load_model('lstm_attack_forecast_model_mleasy.h5')

In [ ]:
model = tf.keras.models.load_model('lstm_attack_forecast_model_mleasy.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter._experimental_lower_tensor_list_ops = False
converter.experimental_new_converter = True
tflite_model = converter.convert()
open("lstm_attack_forecast_model3.tflite", "wb").write(tflite_model)

52152

In [ ]:
model = tf.keras.models.load_model('lstm_attack_forecast_model_mleasy.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [ tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
                                       tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
                                        ]
tflite_model = converter.convert()
with open("best63.tflite", 'wb') as f:
  f.write(tflite_model)

In [ ]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='lstm_attack_forecast_model3.tflite')
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details[0]['shape'])
print(X_test.shape)


# Reshape and convert to float32
#X_test = np.reshape(X_test.astype(np.float32), (20, 30, 1))

# Set input tensor
interpreter.set_tensor(input_details[0]['shape'], X_test)

# Run inference
interpreter.invoke()

# Get output tensor
forecast = interpreter.get_tensor(output_details[0]['index'])

# Inverse transform the forecasted values
forecast = scaler.inverse_transform(forecast)

# Flatten the forecast list
forecast_flat = forecast.flatten()

In [ ]:
import pandas as pd

In [ ]:
def solve_ml_easy(data: pd.DataFrame) -> list:

    # Convert input to DataFrame
    #data = pd.DataFrame(input)
    #data = input
    # Drop rows with NaN values
    #data.dropna(inplace=True)

    # Convert timestamp to datetime
    data['timestamp'] = pd.to_datetime(data['timestamp'])

    # Fill in missing dates with zero attacks
    all_dates = pd.date_range(start=data['timestamp'].min(), end=data['timestamp'].max())
    data.set_index('timestamp', inplace=True)
    data = data.reindex(all_dates, fill_value=0)

    # Smooth out any sudden spikes or outliers
    data['visits'] = data['visits'].rolling(window=7, min_periods=1).mean()

    # Normalize the data
    scaler = MinMaxScaler()
    data['visits_normalized'] = scaler.fit_transform(data['visits'].values.reshape(-1, 1))

    # Prepare data for LSTM
    seq_length = 30
    X_test = []

    for i in range(len(data) - seq_length, len(data)):
        X_test.append(data['visits_normalized'][i-seq_length:i].values)

    X_test = np.array(X_test).astype(np.float32)  # Convert to np.float32

    # #Load the saved model
    # loaded_model = load_model('lstm_attack_forecast_model.h5')

    # #Perform prediction
    # forecast = loaded_model.predict(X_test)


    #X_test = X_test.values.reshape(-1,1,1)





    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path='best63.tflite')
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(input_details[0]['shape'])

    print(input_details[0]['shape'])
    print(X_test.shape)
    #print (X_test)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['shape'], X_test)

    # Run inference
    interpreter.invoke()

    # Get output tensor
    forecast = interpreter.get_tensor(output_details[0]['shape'])

    #Inverse transform the forecasted values
    forecast = scaler.inverse_transform(forecast)
    print (forecast.shape)

    # Flatten the forecast list
    forecast_flat = forecast.flatten()

    # Return the forecasted values for the next 50 days
    return forecast_flat.tolist()

In [ ]:
input = pd.read_csv('series_data.csv')

sol = solve_ml_easy (input)

print (sol)

